In [1]:
from aux_train import *
from aux_plot import *
import plotly.graph_objects as go
from IPython.display import clear_output

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
# config = wandb.config
# config.equation='tranp_equation'
# config.WENO='WENO-Z-DS'

# config.poly_grade=7
# config.seno_ampli=3 
# config.gauss_var=3
# config.seizures=3

sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'validation_error',
      'goal': 'minimize'   
    },
    'parameters': {
        'learnrate': {
            'values': list(range(8))
        },
        'poly_grade': {
            'values': list(range(8))
        },
        'seno_ampli': {
            'values': list(range(4))
        },
        'gauss_var': {
            'values': list(range(4))

        },
        'seizures': {
            'values': list(range(4))
        },
        'optimizer': {
            'values': ['Adam','RMSprop','Adamax','amsgrad','Adadelta','Ftrl']

        },
        'batch_size': {
            'values': [2**i for i in range(2,6)]
        },
        'learning_rate': {
            'distribution': 'log_uniform',
            'min': -5.0,
            'max': -1.0
        },
        'clip_norm': {
            'values': [False,True]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='WENO-DS', entity='silvaneojunior')
#sweep_id = 'silvaneojunior/LSTM Poemas/iuazw8n3'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. learning_rate uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: nf5j7lnm
Sweep URL: https://wandb.ai/silvaneojunior/WENO-DS/sweeps/nf5j7lnm


In [4]:
default_equation='transp_equation'
default_WENO='WENO-Z-DS'

default_poly_grade=7
default_seno_ampli=3 
default_gauss_var=3
default_seizures=3

In [5]:
def f_test(x):
    z = tf.constant(-0.7, dtype=float_pres)
    δ = tf.constant(0.005, dtype=float_pres)
    β = tf.math.log(tf.constant(2.0, dtype=float_pres))/(36.0*(δ**2.0))
    a = tf.constant(0.5, dtype=float_pres)
    α = tf.constant(10, dtype=float_pres)
    
    def G(x, β, z):
        return tf.math.exp(-β*(x-z)**2)
    
    def F(x, α, a):
        return tf.math.sqrt(tf.math.maximum(1-(α**2)*((x-a)**2),0))
    
    f1 = (G(x, β, z-δ) + 4*G(x, β, z) + G(x, β, z+δ))/6 # x in [-0.8, -0.6]
    f2 = 1                                              # x in [-0.4, -0.2]
    f3 = 1-tf.math.abs(10*(x-0.1))                      # x in [ 0.0,  0.2]
    f4 = (F(x, α, a-δ) + 4*F(x, α, a) + F(x, α, a+δ))/6 # x in [ 0.4,  0.6]
    f5 = 0                                              # otherwise
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.8),
        tf.math.less_equal(x, -0.6)
    )
    f1 = f1*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.4),
        tf.math.less_equal(x, -0.2)
    )
    f2 = f2*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.0),
        tf.math.less_equal(x, 0.2)
    )
    f3 = f3*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.4),
        tf.math.less_equal(x, 0.6)
    )
    f4 = f4*tf.cast(tf.where(condition, 1.0, 0.0), dtype=float_pres)
    
    f = f1 + f2 + f3 + f4 + f5
    
    return f

x,fx=create_f_points(f_test=f_test,Δx=0.01)

In [6]:
Sim_Z=simulation(API_Numpy,transp_equation,WENO_Z)
Z_fx=Sim_Z.Sim(fx,2,0.01,0.5,FronteiraPeriodica)

In [7]:
data_temporal,data_spatial,data_base,Δx,Δt,CFL,fronteira,equation=load_dataset(f'datasets/equation={default_equation} polys grau={default_poly_grade} seno_ampli={default_seno_ampli} gauss_var={default_gauss_var} seizure={default_seizures}')

indice = np.arange(data_temporal.shape[0])
np.random.shuffle(indice)
data_x = data_temporal.astype('float64')[indice]
data_y = data_temporal.astype('float64')[indice]

train_x = data_x[:,0:-1]
train_y = data_y[:,1:]

'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None



In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=fx[0],
                    mode='lines',
                    name='Sol. exata'))
fig.add_trace(go.Scatter(x=x, y=Z_fx[0],
                    mode='lines',
                    name='WENO-Z'))

In [9]:
def train():
    clear_output()
    wandb.init()
    config = wandb.config

    # default_config = wandb.config
    # default_config.equation='tranp_equation'
    # default_config.WENO='WENO-Z-DS'

    # default_config.poly_grade=7
    # default_config.seno_ampli=3 
    # default_config.gauss_var=3
    # default_config.seizures=3

    # config = default_config

    run_name = wandb.run.name

    equation_name='transp_equation'
    poly_grade=config.poly_grade
    seno_ampli=config.seno_ampli
    gauss_var=config.gauss_var
    seizures=config.seizures

    
    optimizer=config.optimizer
    batch_size=config.batch_size
    learning_rate=config.learning_rate
    clip_norm=config.clip_norm

    print(f'equation_name={equation_name}')
    print(f'poly_grade={poly_grade}')
    print(f'seno_ampli={seno_ampli}')
    print(f'gauss_var={gauss_var}')
    print(f'seizures={seizures}')
    print(f'optimizer={optimizer}')
    print(f'batch_size={batch_size}')
    print(f'learning_rate={learning_rate}')
    print(f'learning_rate={clip_norm}')

    data_temporal,data_spatial,data_base,Δx,Δt,CFL,fronteira,equation=load_dataset(f'datasets/equation={equation_name} polys grau={poly_grade} seno_ampli={seno_ampli} gauss_var={gauss_var} seizure={seizures}')

    indice = np.arange(data_temporal.shape[0])
    np.random.shuffle(indice)
    data_x = data_temporal.astype('float64')[indice]
    data_y = data_temporal.astype('float64')[indice]

    train_x = data_x[:,0:-1]
    train_y = data_y[:,1:]

    Sim_layer = WENO_temporal_layer(equation,WENO_Z,Δx=Δx,Δt=Δt,fronteira=fronteira,regul_weight=0,ativ_func=tf.nn.sigmoid,p=2,conv_size=5)

    input_x   = keras.layers.Input(train_x.shape[1:], dtype='float64')
    if optimizer=='Adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='RMSprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='Adamax':
        optimizer = keras.optimizers.Adamax(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='amsgrad':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True, clipnorm=clip_norm)
    elif optimizer=='Adadelta':
        optimizer = keras.optimizers.Adadelta(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='Ftrl':
        optimizer = keras.optimizers.Ftrl(learning_rate=learning_rate, clipnorm=clip_norm)
    
    output_x=input_x

    #layers_norm=keras.layers.BatchNormalization()
    #output_x=layers_norm(output_x)
    output_x=keras.layers.TimeDistributed(Sim_layer)(output_x)

    # Criando a rede neural
    Network = keras.Model(input_x, output_x)
    Network.compile(loss=MES_relative(2/Δx), optimizer=optimizer, metrics=['mean_absolute_error'])

    Network.fit(
                train_x                           , # Dados de treino
                train_y                           , # Dados de treino
                batch_size      = batch_size               , # Tamanho do batch
                epochs          = 1              , # Número de epochs
                steps_per_epoch = 1              , # Número de epochs
                shuffle         = True            ,  # Aleatorização dos batchs
                callbacks=[WandbCallback(training_data=(train_x,train_y),log_weights=True,log_gradients=True)]
            )
        #wandb.log({"loss": loss, "epoch": epoch})

    if not(os.path.isdir(f'Modelos treinados/{sweep_id}/')):
        os.mkdir(f'Modelos treinados/{sweep_id}/')
    save_model(Network,Sim_layer.config,f'Modelos treinados/{sweep_id}/{run_name}')

    Δt=0.01*CFL
    total_time=int(2*1/Δt)
    total_time=10
    prev_fx=fx[...]
    count=0
    for i in range(total_time):
        if(i%10==0):
            print(count,end='\r')
        prev_fx=Sim_layer.Sim_step(prev_fx,Δt, 0.01, FronteiraPeriodica)
        count+=Δt

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=x, y=fx[0],
                        mode='lines',
                        name='Sol. exata'))
    fig2.add_trace(go.Scatter(x=x, y=Z_fx[0],
                        mode='lines',
                        name='WENO-Z'))

    fig2.add_trace(go.Scatter(x=x, y=prev_fx[0],
                        mode='lines',
                        name=run_name))

    fig.add_trace(go.Scatter(x=x, y=prev_fx[0],
                        mode='lines',
                        name=run_name))

    wandb.log({'previsão_fx':fig,'previsão_fx_solo':fig2})

    #prev_fx=Sim_layer.Sim(fx,2*(3), 0.01, CFL, FronteiraPeriodica)
    erro=2*tf.math.reduce_mean(tf.abs(prev_fx-fx))/0.01
    
    wandb.log({'validation_error':erro})

#wandb.agent(sweep_id, train)

In [10]:
wandb.agent(sweep_id, train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


equation_name=transp_equation
poly_grade=0
seno_ampli=3
gauss_var=0
seizures=0
optimizer=Adamax
batch_size=32
learning_rate=0.3005320637154497
learning_rate=True
'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None

